# Trabalho 1 - PDI
### Ingrid Dayane da Silva Ferreira (20200154050)

Desenvolva, em uma linguagem de programação de sua escolha, um sistema para abrir, 
exibir, manipular e salvar imagens RGB com 24 bits/pixel (8 bits/componente/pixel). Não 
use bibliotecas ou funções especiais de processamento de imagens. O sistema deve ter a 
seguinte funcionalidade:

1. Correlação m x n com offset, sobre R, G e B. O offset e o filtro (dimensões e valores 
da máscara) devem ser definidos em um arquivo (txt) à parte. Realize testes com os 
filtros Gaussiano 5x5, Sobel horizontal e Sobel vertical, e explique os resultados.
Utilize extensão por zeros. Para visualização do Sobel, aplique valor absoluto seguido 
por expansão de histograma para [0, 255].

In [1]:
def filter(i, f, nome_resultado):
    # importando as bibliotecas necessárias ao processamento
    import cv2
    import numpy as np

    # leitura do txt com as informações do filtro
    with open(f, 'r') as file:  # abrindo o txt do filtro
        m = int(file.readline()) # linhas
        n = int(file.readline()) # colunas
        offset = int(file.readline().strip())
        filtro = [[0] * n for _ in range(m)] # criando uma matriz para colocar o filtro
        linhas = file.readlines()

        for j, line in enumerate(linhas): # colocando o filtro na matriz criada
            f1 = (line.split())
            filtro[j] = [float(valor) for valor in f1]

    img = cv2.imread(i) # lendo a imagem
    img_m, img_n, _ = img.shape # terceira dimensão = bandas (B,G,R)

    img_resultante = np.zeros_like(img) # gerando uma imagem resultante de mesmo tamanho da imagem de entrada

    # Separando os canais
    B = img[:,:,0]
    G = img[:,:,1]
    R = img[:,:,2]

    # criando imagens vazias para alocar o resultado de B, G e R
    B_result = np.zeros_like(B)
    G_result = np.zeros_like(G)
    R_result = np.zeros_like(R)

    for b, b_result in zip([B, G, R],[B_result,G_result,R_result]): # Aplicando o filtro em cada canal
        maximo = np.max(b)
        minimo = np.min(b)
        padded_b = np.pad(b, ((m//2, m//2), (n//2, n//2)),'constant',constant_values=0) # extensão pr zero
        for q in range(img_m): # aplicar o filtro em cada posição da imagem
            for w in range(img_n):
                vizinhanca = padded_b[q:q+m,w:w+n] # recortando a vizinhança para realizar a convolução do filtro (considerando a extensão por zero)
                vizinhanca = ((vizinhanca - minimo)/(maximo - minimo))*255 # aplicando a expansão de histograma com foco na vizinhança
                vizinhanca = vizinhanca.astype('int')
                pos = np.sum(vizinhanca * filtro) + offset #aplicando o filtro na posição (q,w)
                b_result[q,w] = np.absolute(pos) # valor absoluto
                b_result[q,w] = np.clip(b_result[q,w],-255,255) # Delimitando o valor absoluto de -255 a 255
                b_result[q,w] = b_result[q,w].astype('int') #converte para inteiro

    # merge das bandas
    img_resultante[:,:,0] = B_result
    img_resultante[:,:,1] = G_result
    img_resultante[:,:,2] = R_result

    # gera um arquivo
    resultado = cv2.imwrite(nome_resultado, img_resultante)

    return resultado

In [31]:
resultado = filter('Shapes.png','Gaussiano_5x5.txt','Shapes_Gaussiano_5x5.png')

In [32]:
resultado = filter('Shapes.png','Sobel_h.txt','Shapes_Sobel_h.png')

In [33]:
resultado = filter('Shapes.png','Sobel_v.txt','Shapes_Sobel_v.png')

In [34]:
# resultado = filter('Shapes.png','BOX_10x10.txt','Shapes_BOX_10x10.png')

In [35]:
resultado = filter('testpat.tif','Gaussiano_5x5.txt','testpat_Gaussiano_5x5.tif')

In [36]:
resultado = filter('testpat.tif','Sobel_h.txt','testpat_Sobel_h.tif')

In [37]:
resultado = filter('testpat.tif','Sobel_v.txt','testpat_Sobel_v.tif')

2. Filtro pontual a seguir 

![Filtro Pontual](Filtro_Pontual.png)

Esse filtro possui uma função do tipo: <br>
y = 1,992x , para 0 < x <= 128 <br>
y = -1,992x + 508 , para 128 < x < 255

(a) aplicado em RGB

In [66]:
def pontual_filter(i,nome_resultado):
    # importando as bibliotecas necessárias ao processamento
    import cv2
    import numpy as np

    img = cv2.imread(i) # lendo a imagem
    img_m, img_n, _ = img.shape # terceira dimensão = bandas (B,G,R)

    img_resultante = np.zeros_like(img) # gerando uma imagem resultante de mesmo tamanho da imagem de entrada

    # Separando os canais
    B = img[:,:,0]
    G = img[:,:,1]
    R = img[:,:,2]

    # criando imagens vazias para alocar o resultado de B, G e R
    B_result = np.zeros_like(B)
    G_result = np.zeros_like(G)
    R_result = np.zeros_like(R)

    for b, b_result in zip([B,G,R],[B_result,G_result,R_result]): # Aplicando o filtro em cada canal
        for q in range(img_m): # aplicar o filtro em cada posição da imagem
            for w in range(img_n):
                # As fórmulas foram calculadas no GeoGebra
                if b[q,w] <= 128:
                    b_result[q,w] = 1.992*b_result[q,w] 
                else:
                    b_result[q,w] = np.clip((-1.992*b_result[q,w] + 508),0,255)

    # # merge das bandas
    img_resultante[:,:,0] = B_result
    img_resultante[:,:,1] = G_result
    img_resultante[:,:,2] = R_result

    nome_B = nome_resultado + '_B.tif'
    nome_G = nome_resultado + '_G.tif'
    nome_R = nome_resultado + '_R.tif'

    # gera um arquivo
    resultado = cv2.imwrite(nome_resultado, img_resultante)
    resultado = cv2.imwrite(nome_B, img_resultante[:,:,0])
    resultado = cv2.imwrite(nome_G, img_resultante[:,:,1])
    resultado = cv2.imwrite(nome_R, img_resultante[:,:,2])

    return resultado

In [67]:
resultado = pontual_filter('testpat.tif','testpat_Pontal.tif')

In [68]:
resultado = pontual_filter('color.png','color_Pontal.png')

(b) aplicado à banda Y do YIQ, com posterior conversão a RGB

In [69]:
def rgb2yiq(i, nome_resultado):
    # importando as bibliotecas necessárias ao processamento
    import cv2
    import numpy as np

    img = cv2.imread(i) # lendo a imagem
    img_m, img_n, _ = img.shape # terceira dimensão = bandas (B,G,R)

    img_resultante = np.zeros_like(img) # gerando uma imagem resultante de mesmo tamanho da imagem de entrada

    # Separando os canais
    B = img[:,:,0]
    G = img[:,:,1]
    R = img[:,:,2]

    # criando imagens vazias para alocar o resultado de B, G e R
    Y = np.zeros_like(B)
    I = np.zeros_like(G)
    Q = np.zeros_like(R)

    for q in range(img_m): # aplicar o filtro em cada posição da imagem
        for w in range(img_n):
            Y[q,w] = (0.299 * R[q,w]) + (0.587 * G[q,w]) + (0.114 * B[q,w])
            I[q,w] = (0.596 * R[q,w]) - (0.274 * G[q,w]) - (0.322 * B[q,w])
            Q[q,w] = (0.211 * R[q,w]) - (0.523 * G[q,w]) + (0.312 * B[q,w])

    for q in range(img_m): # aplicar o filtro em cada posição da imagem
        for w in range(img_n):
            # As fórmulas foram calculadas no GeoGebra
            if Y[q,w] <= 128:
                Y[q,w] = 1.992*Y[q,w] 
            else:
                Y[q,w] = np.clip((-1.992*Y[q,w] + 508),0,255)

    # # merge das bandas
    img_resultante[:,:,0] = Y
    img_resultante[:,:,1] = I
    img_resultante[:,:,2] = Q

    nome_B = nome_resultado + '_Y.png'
    nome_G = nome_resultado + '_I.png'
    nome_R = nome_resultado + '_Q.png'

    # gera um arquivo
    resultado = cv2.imwrite(nome_resultado, img_resultante)
    resultado = cv2.imwrite(nome_B, img_resultante[:,:,0])
    resultado = cv2.imwrite(nome_G, img_resultante[:,:,1])
    resultado = cv2.imwrite(nome_R, img_resultante[:,:,2])

    return resultado

In [70]:
resultado = rgb2yiq('testpat.tif','testpat_YIQ.tif')

In [71]:
resultado = rgb2yiq('color.png','color_YIQ.png')

In [72]:
def yiq2rgb(i, nome_resultado):
    # importando as bibliotecas necessárias ao processamento
    import cv2
    import numpy as np

    img = cv2.imread(i) # lendo a imagem
    img_m, img_n, _ = img.shape # terceira dimensão = bandas (Y,I,Q)

    img_resultante = np.zeros_like(img) # gerando uma imagem resultante de mesmo tamanho da imagem de entrada

    # Separando os canais
    Y = img[:,:,0]
    I = img[:,:,1]
    Q = img[:,:,2]

    # criando imagens vazias para alocar o resultado de B, G e R
    B = np.zeros_like(Y)
    G = np.zeros_like(Y)
    R = np.zeros_like(Y)

    for q in range(img_m): # aplicar o filtro em cada posição da imagem
        for w in range(img_n):
            B[q,w] = (Y[q,w]) + (0.956 * I[q,w]) + (0.621 * Q[q,w])
            G[q,w] = (Y[q,w]) - (0.272 * I[q,w]) - (0.647 * Q[q,w])
            R[q,w] = (Y[q,w]) - (1.106 * I[q,w]) + (1.703 * Q[q,w])

    # # merge das bandas
    img_resultante[:,:,0] = B
    img_resultante[:,:,1] = G
    img_resultante[:,:,2] = R

    nome_B = nome_resultado + '_B.png'
    nome_G = nome_resultado + '_G.png'
    nome_R = nome_resultado + '_R.png'

    # gera um arquivo
    resultado = cv2.imwrite(nome_resultado, img_resultante)
    resultado = cv2.imwrite(nome_B, img_resultante[:,:,0])
    resultado = cv2.imwrite(nome_G, img_resultante[:,:,1])
    resultado = cv2.imwrite(nome_R, img_resultante[:,:,2])

    return resultado

In [73]:
resultado = yiq2rgb('testpat_YIQ.tif','testpat_volta_RGB.tif')

In [74]:
resultado = yiq2rgb('color_YIQ.png','color_volta_RGB.png')